# Setup
Installation und Import von benötigten sog. *libraries*

Für Euch:
Bitte öffnet folgenden Link:

# https://yopad.eu/p/Amadis_digital

In [ ]:
!python -m spacy download "de_core_news_lg"
%pip install pyldavis
%pip install gensim

In [1]:
import spacy
from pathlib import Path
import gensim
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
from pyLDAvis import gensim_models as gsv
from constants import STOPWORDS as stops
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from collections import Counter
import nltk
import math
import pandas as pd
import itertools
from helpers import leven_worker, pre_cleaning
%matplotlib inline

Die *library* "spacy" nimmt uns einen wichtigen Teil der Arbeit ab: Die sog. *tokenization* und andere, wichtige Aspekte bei der Vorbereitung des Korpus.

spaCy ist eine sog. *NLP-Library* und stellt *pipelines* bereit. Vor- oder selbsttrainierte Modelle erlauben *tokenization*, *stemming*, *Lemmatisierung*, *named entity recognition* uvm.

In [2]:
nlp = spacy.load("de_core_news_lg")
nlp.max_length = 1500000

/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(s

In [3]:
input_files = Path("./input/").rglob("*.txt")

In [4]:
full_corpus = {}

for i in input_files:
    with open(i, "r+") as f:
        txt = f.read()
    txt = pre_cleaning(txt)
    book_name = str(i).split("/")[-1]
    book_name = book_name.split(".")[0]
    full_corpus[book_name] = txt
print(len(full_corpus))

6


## Grundlagen Python

#### Wichtige Datentypen

Dictionary (dict): key:value.
{}

List (list): value, value, value...
[]

String (str): Das hier ist ein String, aber nur wenn "er in Anfürhungszeichen" 
steht, dabei ist es egal, ob "doppelte" oder 'einfache' Anführungszeichen, 
sie dürfen nur "nie gemischt werden'

Integer (int): 1

Gleitkommazahl (float): 3.1415


#### Quiz:

Was ist das:

{"Basel": ["Basel SBB", "Basel Bad"]}

basel = ["Basel SBB", "Basel Bad"]

["Astrid Lindgren", "Jón Jónsson", 2, "5.7"]

# Teil 1: Topic Modelling

Wie der Name schon sagt: Trainiert ein Modell für die *topics* also Themen einer Dokument-Sammlung.

<div>
<img src="imgs/top_mod.png" width="800"/>
</div>

Quelle: Ghanoum, Tarek: "Topic Modelling in Python with spaCy and Gensim". https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf

In [5]:
by_chapters = {}

for k, v in full_corpus.items():
    chaps = v.split("Capitel")
    chap_num = 0
    for i in chaps:
        curr_chap = f"{k}-{chap_num}-Kap"
        by_chapters[curr_chap] = i
        chap_num += 1
print(len(by_chapters))

398


In [6]:
docs_list: list[list[str]] = []
for i in nlp.pipe(by_chapters.values(), n_process=7):
    itoks = [token.lemma_.lower() for token in i if token.lower_ not in stops and "\n" not in token.lower_ and not token.is_stop]
    docs_list.append(itoks)

/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(s

In [7]:
bigram_phrases = gensim.models.Phrases(docs_list, min_count=5, threshold=2)
trigram_phrases = gensim.models.Phrases(bigram_phrases[docs_list], threshold=2)
bigrams = gensim.models.phrases.Phraser(bigram_phrases)
trigrams = gensim.models.phrases.Phraser(trigram_phrases)
docs_list = trigrams[bigrams[docs_list]]
corpus_dictionary = gensim.corpora.Dictionary(docs_list)
analysis_corpus = [corpus_dictionary.doc2bow(i) for i in docs_list]

### Wie viele Topics?

<div>
<img src="imgs/model_coherence.png" width="800">
</div

In [ ]:
topics = []
score = []
for i in range(1, 40,1):
   lda_model = LdaMulticore(corpus=analysis_corpus,
               id2word=corpus_dictionary,
               iterations=75,
               num_topics=i,
               workers = 7,
               passes=12,
               random_state=100)
   cm = CoherenceModel(model=lda_model,
                       corpus=analysis_corpus,
                       dictionary=corpus_dictionary,
                       coherence='c_v',
                       texts=docs_list)
   topics.append(i)
   score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')
_=plt.ylabel('Coherence Score')
plt.show()

### Model training

In [10]:

lda_model = LdaMulticore(corpus=analysis_corpus,
                                              id2word=corpus_dictionary,
                                              iterations=50,
                                              num_topics=5,
                                              workers = 7,
                                              passes=10)


In [ ]:
topics_to_show = 10
words_to_show = 10
lda_model.print_topics(10, 10)

In [ ]:
docs_no = []
tops = []
all_chaps = list(by_chapters.keys())
for index, chap in enumerate(all_chaps):
    print(index, chap)
    docs_no.append(chap)
    tops.append(lda_model[analysis_corpus][index])

pd_dict = {"Kapitel": docs_no, "Topics": tops}

In [ ]:
chap_top_df = pd.DataFrame(pd_dict)
pd.set_option('display.max_rows', 400)
chap_top_df

In [ ]:
top_nos = []
for index, row in chap_top_df.iterrows():
    num_tops = len(row['Topics'])
    top_nos.append(num_tops)
chap_top_df['Bandbreite'] = top_nos
chap_top_df.sort_values(by=["Bandbreite"], ascending=False)

In [ ]:
top_dists = Counter(top_nos)
plt.bar(top_dists.keys(), top_dists.values())
plt.show()

In [11]:
lda_display = gsv.prepare(lda_model, analysis_corpus, corpus_dictionary)
pyLDAvis.display(lda_display)

/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/kraus0

#### Concordances

In [12]:
searchword = "könig"
locations: list[tuple[int, int]] = []
for doc in enumerate(docs_list):
    doc_no, doc_toks = doc
    for toks in enumerate(doc_toks):
        tok_no, tok = toks
        if tok == searchword:
            locations.append((doc_no, tok_no))

for location in locations:
    doc_loc = location[0]
    tok_scope_from = location[1]-5
    tok_scope_to = location[1]+5
    print(docs_list[doc_loc][tok_scope_from:tok_scope_to])



['j.', 'f.', 'w.', 'beschreiben', 'gewiß', 'könig', 'mithridate', 'verfolget', 'jn', 'jugend']
['wa', 'funden', 'kraut', 'giftwend', 'genent', 'könig', 'gewent', 'darm', 'gebeitzt', 'magen']
['bluthund', 'tyrann', 'wütrich', 'kurtzumb', 'bern', 'könig', 'dietrich', 'türck', 'tartarn', 'heyden']
['zeit', 'regieren', 'königreich', 'saba', 'mohr', 'könig', 'name', 'magadan', 'gewonheit', 'derselbig']
['war', 'gantze_königreich', 'saba', 'lautbar', 'lüste', 'könig', 'frembd', 'gefang', 'empfienge', 'bevorab']
['knabe', 'dreyer', 'jar_alt', 'hend', 'grüßt', 'könig', 're', 'gewonheit', 'allergnedister', 'herr_könig']
['verstund', 'mandatar', 'unangeseh', 'gelehrt', 'vergleichen', 'könig', 'erfreuet', 'schonen', 'verehrung', 'od']
['pat', 'streich', 'kibel', 'schluge', 'wünschet', 'könig', 'spieß', 'bote', 'verwunden', 'thier']
['zoge', 'spieß', 'ge_walt', 'fest', 'umbgriffe', 'könig', 'warff', 'boden', 'juncker', 'ersehen']
['krafft', 'kopff', 'zurspalten', '  ', 'vermeinte', 'könig', 'auffz

#### Frequenzanalyse

In [16]:
searchword = "keiser"
count = 0
for doc in docs_list:
    for tok in doc:
        if tok == searchword:
            count += 1
print(count)

44


# Teil 2: Stilometrie

In [17]:
author_dict: dict[str, list[str]] = {"Montalvo": ["1__Buch", "4__Buch"], "Silva": ["7__Buch", "10__Buch"], "Roseo": ["19__Buch"], "Unknown": ["22__Buch"] }
translator_dict: dict[str, str] = {"unknown": ["1__Buch", "10__Buch", "19__Buch"], "I.VV.V.L":["4__Buch"], "J.W.V.L.":["7__Buch"], "E.(?)B.D.J":["22__Buch"]}
known = ["Montalvo", "Silva", "Roseo"]

## Basic: Wort- und Satzlängen

### Wortlängen

In [ ]:
whole_books_clean: dict[str, list[str]] = {}
for k, v in full_corpus.items():
    i = nlp(v)
    itoks = [token.lemma_.lower() for token in i if token.lower_ not in stops and "\n" not in token.lower_ and not token.is_stop]
    whole_books_clean[k] = itoks

In [ ]:
length_by_book = {}
for k, v in whole_books_clean.items():
    token_lengths = [len(token) for token in v]
    freq_dist = Counter(token_lengths)
    length_by_book[k] = freq_dist


In [ ]:
for k, v in length_by_book.items():
    plt.bar(v.keys(), v.values())
    plt.title(f"{k}")
    plt.show()

### Satzlängen

In [ ]:
books_by_sents: dict[str, list[str]] = {}
for k, v in full_corpus.items():
    i = nlp(v)
    sents = [sent.text for sent in i.sents]
    books_by_sents[k] = sents

In [ ]:
length_by_book = {}
for k, v in books_by_sents.items():
    sent_lengths = [len(sent.split()) for sent in v]
    freq_dist = Counter(sent_lengths)
    length_by_book[k] = freq_dist

In [ ]:
for k, v in length_by_book.items():
    plt.bar(v.keys(), v.values())
    plt.title(f"{k}")
    plt.show()

## Advanced: Burrows Delta

<div>
<img src="imgs/delta.png" width="800">
</div>
Quelle: Büttner et al: "»Delta« in der stilometrischen Autorschaftsattribution", Zeitschrift für digitale Geisteswissenschaften, http://dx.doi.org/10.17175/2017_006

In [18]:
whole_books_burrows: dict[str, list[str]] = {}
for k, v in full_corpus.items():
    i = nlp(v)
    itoks = [token.lemma_.lower() for token in i if "\n" not in token.lower_ and "--" not in token.lower_]
    whole_books_burrows[k] = itoks

In [19]:
complete_corpus = []
for k, v in whole_books_burrows.items():
    complete_corpus.extend(v)
len(complete_corpus)

1121948

In [20]:
whole_corpus_freq_dist = list(nltk.FreqDist(complete_corpus).most_common(30))
whole_corpus_freq_dist[ :10 ]

[(' ', 117306),
 ('der', 71464),
 ('--', 36750),
 ('und', 33847),
 ('sein', 18834),
 ('sie', 18535),
 ('zu', 18023),
 ('in', 17007),
 ('er', 15992),
 ('von', 14359)]

In [21]:
features = [word for word,freq in whole_corpus_freq_dist]
feature_freqs = {}

for i in known:
    feature_freqs[i] = {}
    personal_complete = []

    for ii in author_dict[i]:
        personal_complete.extend(whole_books_burrows[ii])

    token_count = len(personal_complete)
    for feature in features:
        presence = personal_complete.count(feature)
        feature_freqs[i][feature] = presence / token_count

In [22]:
corpus_features = {}

for feature in features:
    corpus_features[feature] = {}

    feature_avg = 0
    for person in known:
        feature_avg += feature_freqs[person][feature]
    feature_avg /= len(known)
    corpus_features[feature]["Mean"] = feature_avg

    feature_stdev = 0
    for person in known:
        diff = feature_freqs[person][feature] - corpus_features[feature]["Mean"]
        feature_stdev += diff*diff
    feature_stdev /= (len(known) - 1)
    feature_stdev = math.sqrt(feature_stdev)
    corpus_features[feature]["StdDev"] = feature_stdev

In [23]:
feature_zscores = {}
for person in known:
    feature_zscores[person] = {}
    for feature in features:

        feature_val = feature_freqs[person][feature]
        feature_mean = corpus_features[feature]["Mean"]
        feature_stdev = corpus_features[feature]["StdDev"]
        feature_zscores[person][feature] = ((feature_val-feature_mean) /
                                            feature_stdev)


In [24]:
for i in author_dict["Unknown"]:
    unkown_text = whole_books_burrows[i]

    unkown_count = len(unkown_text)
    unkown_freqs = {}
    for feature in features:
        presence = unkown_text.count(feature)
        unkown_freqs[feature] = presence / unkown_count
    
    unkown_zscores = {}
    for feature in features:
        feature_val = unkown_freqs[feature]
        feature_mean = corpus_features[feature]["Mean"]
        feature_stdev = corpus_features[feature]["StdDev"]
        unkown_zscores[feature] = (feature_val - feature_mean) / feature_stdev

In [25]:
for person in known:
    delta = 0
    for feature in features:
        delta += math.fabs((unkown_zscores[feature] -
                            feature_zscores[person][feature]))
    delta /= len(features)
    print(f"Delta score for candidate {person} is {delta}")


Delta score for candidate Montalvo is 1.1018143768323694
Delta score for candidate Silva is 0.8937642209658405
Delta score for candidate Roseo is 0.9620684580422975


## Bonus: Text reuse

### Simple trick?

In [ ]:
all_sents = [sent for sents in books_by_sents.values() for sent in sents if len(sent.split()) > 4]

In [ ]:
reoccurences = Counter(all_sents)
most_freq_sents = []
most_freq_counts = []
for sent in reoccurences:
    if reoccurences[sent] > 1:
        most_freq_sents.append(sent)
        most_freq_counts.append(int(reoccurences[sent]))

In [ ]:
df_dict = {"Sentence": most_freq_sents, "Count": most_freq_counts}
freqs_df = pd.DataFrame(df_dict)
freqs_df.sort_values(by=["Count"], ascending=False)

### Levenshtein

In [ ]:
sent_combinations = itertools.combinations(all_sents[:6000], 2)
matches_dict = {}
matches_list = []
for i in leven_worker(sent_combinations):
    matches_dict[i[0]] = i[1]
    matches_list.append(i[0])
    print(i)


In [ ]:
len(matches_list)